In [1]:
import logging
import sys
from pathlib import Path
import itertools

import click
import numpy as np
import pandas as pd
import rasterra as rt
from rra_tools import jobmon
from scipy.special import expit
import xarray as xr
import tqdm
import rasterra as rt
from rasterio.features import rasterize
import geopandas as gpd

from spatial_temp_cgf import paths
from spatial_temp_cgf.training_data_prep import income_funcs
from spatial_temp_cgf.training_data_prep.location_mapping import load_fhs_lsae_mapping
from spatial_temp_cgf import cli_options as clio
from spatial_temp_cgf.data import DEFAULT_ROOT, ClimateMalnutritionData

In [2]:
RASTER_TEMPLATE_PATH = Path('/mnt/team/rapidresponse/pub/population/data/01-raw-data/other-gridded-pop-projects/global-human-settlement-layer/1km_template.tif')
SHAPE_PATH = Path('/mnt/team/rapidresponse/pub/population/data/02-processed-data/ihme/lbd_admin2.parquet')
LDIPC_FILEPATH = Path('/share/resource_tracking/forecasting/poverty/GK_2024_income_distribution_forecasts/income_forecasting_through2100_admin2_final_nocoviddummy_intshift/admin2_ldipc_estimates.csv')

In [34]:
output_dir = Path(DEFAULT_ROOT)
measure = 'stunting'
model_version = "2024_06_30.07"
cmip6_scenario = 'ssp126'
year = 2025

In [35]:
cm_data = ClimateMalnutritionData(output_dir / measure)

In [24]:
raster_template = rt.load_raster(RASTER_TEMPLATE_PATH)
a2 = gpd.read_parquet(SHAPE_PATH)

In [25]:
ldi = pd.read_csv(LDIPC_FILEPATH)
national_mean = ldi.groupby(['year_id', 'national_ihme_loc_id', 'population_percentile']).ldipc.transform('mean')
null_mask = ldi.ldipc.isnull()
ldi.loc[null_mask, 'ldipc'] = national_mean.loc[null_mask]
ldi['ldi_pc_pd'] = ldi['ldipc'] / 365.25
ldi = ldi.groupby(['year_id', 'location_id']).ldi_pc_pd.mean().reset_index()
polys = a2.loc[a2.loc_id.isin(ldi.location_id.unique()), ['loc_id', 'geometry']].rename(columns={'loc_id': 'location_id'}).set_index('location_id').geometry

year_ldi = ldi[ldi.year_id == 2000].set_index('location_id').ldi_pc_pd
shapes = [(t.geometry, t.ldi_pc_pd) for t in pd.concat([year_ldi, polys.sort_index()], axis=1).itertuples()]
arr = rasterize(
    shapes, 
    out=np.zeros_like(raster_template), 
    transform=raster_template.transform, 
)
r = rt.RasterArray(arr, transform=raster_template.transform, crs=raster_template.crs, no_data_value=np.nan)

In [36]:
models = cm_data.load_model_family(model_version)

In [47]:
m = models[0]

In [48]:
m['model'].var_info

{'intercept': {},
 'mean_temperature': {'feature_range': (-10.310000000000002,
   31.21999999999997)},
 'ldi_pc_pd': {'feature_range': (0.00021000055729025876, 179.61527230987207)}}

In [44]:
from sklearn.preprocessing import MinMaxScaler

In [49]:
s = MinMaxScaler()

In [54]:
s.set_params(**m['model'].var_info['ldi_pc_pd'])

MinMaxScaler(feature_range=(0.00021000055729025876, 179.61527230987207))

In [52]:
s.transform(r)

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.